In [49]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [50]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [63]:
LOCAL = False

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [64]:
%%capture pwd
!pwd

In [65]:
# uid = "2021-05-05-which-firms-and-industries-have-been-most-affected-by-covid-update"  # article unique ID
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
uid, height, width

('2021-08-04-where-are-the-uks-levelling-up-funds-most-needed', 300, 500)

# Fig 1

In [66]:
df = pd.read_csv("raw/tracktheeconomyacuk_england_index_db.csv")

In [67]:
f = "fig1_imd"
f1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,local_authorities,y_axis_allcat_rank,x_axis_imd_rank,lad_id,country,levelling_up_categorisation,region,supergroup_name,quadrant,label_2_highest_lowest
0,Haringey,11,37,E09000014,England,2,London,London Cosmopolitan,quadrant 1,lab
1,Great Yarmouth,26,24,E07000145,England,1,East,Countryside Living,quadrant 1,lab
2,Luton,8,52,E06000032,England,1,East,Ethnically Diverse Metropolitan Living,quadrant 1,nolab
3,Tendring,29,32,E07000076,England,1,East,Countryside Living,quadrant 1,nolab
4,Barking and Dagenham,58,5,E09000002,England,1,London,Ethnically Diverse Metropolitan Living,quadrant 1,nolab


In [ ]:
selection = alt.selection_multi(fields=["supergroup_name"], bind="legend")
selection2 = alt.selection_multi(fields=["region"], bind="legend")
base = (
    alt.Chart(f1)
    .encode(
        tooltip="tooltip:N",
        x=alt.X(
            "x_axis_imd_rank:Q",
            axis=None,
            scale=alt.Scale(domain=[0, 300]),
        ),
        y=alt.Y(
            "y_axis_allcat_rank:Q",
            sort=[],
            axis=None,
            scale=alt.Scale(domain=[0, 300]),
        ),
        fill=alt.Color(
            "supergroup_name:N",
            scale=alt.Scale(
                range=[
                    colors["eco-red"],
                    "white",
                    colors["eco-mid-blue"],
                    "white",
                    colors["eco-green"],
                    "white",
                    colors["eco-gray"],
                    "white",
                ],
            ),
        ),
        color=alt.Color(
            "supergroup_name:N",
            legend=alt.Legend(title="", labelLimit=400, orient="bottom", columns=2),
            scale=alt.Scale(
                range=[
                    scale_lightness(colors["eco-red"], 0.7),
                    scale_lightness(colors["eco-red"], 0.7),
                    scale_lightness(colors["eco-mid-blue"], 0.7),
                    scale_lightness(colors["eco-mid-blue"], 0.7),
                    scale_lightness(colors["eco-green"], 0.7),
                    scale_lightness(colors["eco-green"], 0.7),
                    scale_lightness(colors["eco-gray"], 0.7),
                    scale_lightness(colors["eco-gray"], 0.7),
                ],
            ),
        ),
        shape=alt.Shape(
            "region:N",
            legend=alt.Legend(
                title="",
                labelLimit=170,
                orient="bottom",
                columns=3,
                symbolFillColor=colors["eco-gray"],
                symbolStrokeColor=scale_lightness(colors["eco-gray"], 0.7),
                offset=30,
            ),
            scale=alt.Scale(
                range=[
                    "triangle-right",
                    "cross",
                    "circle",
                    "triangle-up",
                    "diamond",
                    "triangle-down",
                    "triangle-left",
                    "M0,.5L.6,.8L.5,.1L1,-.3L.3,-.4L0,-1L-.3,-.4L-1,-.3L-.5,.1L-.6,.8L0,.5Z",
                    "square",
                ]
            ),
        ),
        opacity=alt.condition(selection | selection2, alt.value(0.8), alt.value(0.1)),
    )
    .transform_calculate(
        tooltip="datum.region+' 🏨 '+datum.local_authorities+' 📊 X: '+datum.x_axis_imd_rank+'. Y: '+datum.y_axis_allcat_rank+'.'"
    )
    .add_selection(selection)
    .add_selection(selection2)
)
points1 = base.mark_point(size=90).encode()
label1 = (
    alt.Chart(pd.DataFrame([{"x": 0, "y": -10, "t": "⬅ Increasing deprivation"}]))
    .mark_text(size=11, color=colors["eco-gray"], align="left")
    .encode(x="x", y="y", text="t")
)
label2 = (
    alt.Chart(
        pd.DataFrame([{"x": -10, "y": 0, "t": "⬅ Increasing COVID economic impact"}])
    )
    .mark_text(size=11, color=colors["eco-gray"], align="left", angle=270)
    .encode(x="x", y="y", text="t")
)
label3 = (
    alt.Chart(pd.DataFrame([{"x": 310, "y": 320, "t": "Decreasing deprivation ➡"}]))
    .mark_text(size=11, color=colors["eco-gray"], align="right")
    .encode(x="x", y="y", text="t")
)
label4 = (
    alt.Chart(
        pd.DataFrame([{"x": 320, "y": 310, "t": "Decreasing COVID economic impact ➡"}])
    )
    .mark_text(size=11, color=colors["eco-gray"], align="right", angle=270)
    .encode(x="x", y="y", text="t")
)
label5 = (
    alt.Chart(pd.DataFrame([{"x": 178, "y": -65, "t": "⬆ click to filter"}]))
    .mark_text(size=10, color=colors["eco-gray"], align="left")
    .encode(x="x", y="y", text="t")
)
line1 = (
    alt.Chart(pd.DataFrame([{"x": 155}]))
    .mark_rule(strokeDash=[5, 3], color=colors["eco-gray"])
    .encode(x="x:Q")
)
line2 = (
    alt.Chart(pd.DataFrame([{"x": 155}]))
    .mark_rule(strokeDash=[5, 3], color=colors["eco-gray"])
    .encode(y="x:Q")
)
area1 = (
    alt.Chart(
        pd.DataFrame([{"x": 155, "y": 0, "y2": 311}, {"x": 311, "y": 0, "y2": 311}])
    )
    .mark_area(
        opacity=0.15,
        color=alt.Gradient(
            gradient="linear",
            stops=[
                alt.GradientStop(color="white", offset=0),
                alt.GradientStop(color=colors["eco-gray"], offset=0.3),
            ],
            x1=1,
            x2=0,
            y1=1,
            y2=1,
        ),
    )
    .encode(x="x:Q", y="y:Q", y2="y2:Q")
)
area2 = (
    alt.Chart(
        pd.DataFrame([{"x": 0, "y": 155, "y2": 330}, {"x": 311, "y": 155, "y2": 330}])
    )
    .mark_area(
        opacity=0.15,
        color=alt.Gradient(
            gradient="linear",
            stops=[
                alt.GradientStop(color="white", offset=0),
                alt.GradientStop(color=colors["eco-gray"], offset=0.3),
            ],
            x1=1,
            x2=1,
            y1=0,
            y2=1,
        ),
    )
    .encode(x="x:Q", y="y:Q", y2="y2:Q")
)
text1 = (
    base.mark_text()
    .encode(text="local_authorities:N")
    .transform_filter("datum.label_2_highest_lowest=='lab'")
)
layer1 = (
    (
        (
#             area1 
#             + area2
            points1
            + line1
            + line2
            + label1
            + label2
            + label3
            + label4
            + label5
        ).properties(height=400, width=600)
    )
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "w").write(readme)
layer1

# Fig 2

In [ ]:
df['levelling_up_categorisation']=df['levelling_up_categorisation'].replace(1,'1: Most need').replace(2,'2: Mid need').replace(3, '3: Least need')

In [ ]:
f = "fig2_category"
f2 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f2.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f2 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

In [ ]:
selection = alt.selection_multi(fields=["levelling_up_categorisation"], bind="legend")
base = (
    alt.Chart(f2)
    .encode(
        tooltip="tooltip:N",
        x=alt.X(
            "x_axis_imd_rank:Q",
            axis=None,
            scale=alt.Scale(domain=[0, 300]),
        ),
        y=alt.Y(
            "y_axis_allcat_rank:Q",
            sort=[],
            axis=None,
            scale=alt.Scale(domain=[0, 300]),
        ),
        fill=alt.Color(
            "levelling_up_categorisation:N",
            scale=alt.Scale(
                range=[
                    colors["eco-red"],
                    colors["eco-mid-blue"],
                    colors["eco-green"],
                ],
            ),
        ),
        color=alt.Color(
            "levelling_up_categorisation:N",
            legend=alt.Legend(title="", labelLimit=400, orient="none",direction='horizontal',legendY=418,legendX=350,labelColor=colors['eco-gray']),
            scale=alt.Scale(
                range=[
                    scale_lightness(colors["eco-red"], 0.7),
                    scale_lightness(colors["eco-mid-blue"], 0.7),
                    scale_lightness(colors["eco-green"], 0.7),
                ],
            ),
        ),
        opacity=alt.condition(selection, alt.value(0.8), alt.value(0.1)),
    )
    .transform_calculate(
        tooltip="datum.region+' 🏨 '+datum.local_authorities+' 📊 X: '+datum.x_axis_imd_rank+'. Y: '+datum.y_axis_allcat_rank+'.'"
    )
    .add_selection(selection)
)
points1 = base.mark_point(size=90).encode()
label1 = (
    alt.Chart(pd.DataFrame([{"x": 0, "y": -10, "t": "⬅ Increasing deprivation"}]))
    .mark_text(size=11, color=colors["eco-gray"], align="left")
    .encode(x="x", y="y", text="t")
)
label2 = (
    alt.Chart(
        pd.DataFrame([{"x": -10, "y": 0, "t": "⬅ Increasing COVID economic impact"}])
    )
    .mark_text(size=11, color=colors["eco-gray"], align="left", angle=270)
    .encode(x="x", y="y", text="t")
)
label3 = (
    alt.Chart(pd.DataFrame([{"x": 310, "y": 320, "t": "Decreasing deprivation ➡"}]))
    .mark_text(size=11, color=colors["eco-gray"], align="right")
    .encode(x="x", y="y", text="t")
)
label4 = (
    alt.Chart(
        pd.DataFrame([{"x": 320, "y": 310, "t": "Decreasing COVID economic impact ➡"}])
    )
    .mark_text(size=11, color=colors["eco-gray"], align="right", angle=270)
    .encode(x="x", y="y", text="t")
)
label5 = (
    alt.Chart(pd.DataFrame([{"x": 105, "y": -18, "t": "Levelling Up Fund category"}]))
    .mark_text(size=11, color=colors["eco-gray"], align="left")
    .encode(x="x", y="y", text="t")
)
line1 = (
    alt.Chart(pd.DataFrame([{"x": 155}]))
    .mark_rule(strokeDash=[5, 3], color=colors["eco-gray"])
    .encode(x="x:Q")
)
line2 = (
    alt.Chart(pd.DataFrame([{"x": 155}]))
    .mark_rule(strokeDash=[5, 3], color=colors["eco-gray"])
    .encode(y="x:Q")
)
area1 = (
    alt.Chart(
        pd.DataFrame([{"x": 155, "y": 0, "y2": 311}, {"x": 311, "y": 0, "y2": 311}])
    )
    .mark_area(
        opacity=0.15,
        color=alt.Gradient(
            gradient="linear",
            stops=[
                alt.GradientStop(color="white", offset=0),
                alt.GradientStop(color=colors["eco-gray"], offset=0.3),
            ],
            x1=1,
            x2=0,
            y1=1,
            y2=1,
        ),
    )
    .encode(x="x:Q", y="y:Q", y2="y2:Q")
)
area2 = (
    alt.Chart(
        pd.DataFrame([{"x": 0, "y": 155, "y2": 330}, {"x": 311, "y": 155, "y2": 330}])
    )
    .mark_area(
        opacity=0.15,
        color=alt.Gradient(
            gradient="linear",
            stops=[
                alt.GradientStop(color="white", offset=0),
                alt.GradientStop(color=colors["eco-gray"], offset=0.3),
            ],
            x1=1,
            x2=1,
            y1=0,
            y2=1,
        ),
    )
    .encode(x="x:Q", y="y:Q", y2="y2:Q")
)
text1 = (
    base.mark_text()
    .encode(text="local_authorities:N")
    .transform_filter("datum.label_2_highest_lowest=='lab'")
)
layer1 = (
    (
        (
#             area1 
#             + area2
            points1
            + line1
            + line2
            + label1
            + label2
            + label3
            + label4
            + label5
        ).properties(height=400, width=600)
    )
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

# Fig 3

In [ ]:
f = "fig3_faceted"
f3 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f3.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f3 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

In [ ]:
selection = alt.selection_multi(fields=["supergroup_name"], bind="legend")
selection2 = alt.selection_multi(fields=["region"], bind="legend")
base = (
    alt.Chart(f3)
    .encode(
        tooltip="tooltip:N",
        x=alt.X(
            "x_axis_imd_rank:Q",
            axis=None,
            scale=alt.Scale(domain=[-20, 330]),
        ),
        y=alt.Y(
            "y_axis_allcat_rank:Q",
            sort=[],
            axis=None,
            scale=alt.Scale(domain=[-20, 330]),
        ),
        fill=alt.Color(
            "supergroup_name:N",
            scale=alt.Scale(
                range=[
                    colors["eco-red"],
                    "white",
                    colors["eco-mid-blue"],
                    "white",
                    colors["eco-green"],
                    "white",
                    colors["eco-gray"],
                    "white",
                ],
            ),
        ),
        color=alt.Color(
            "supergroup_name:N",
            legend=alt.Legend(title="", labelLimit=400, orient="bottom", columns=2),
            scale=alt.Scale(
                range=[
                    scale_lightness(colors["eco-red"], 0.7),
                    scale_lightness(colors["eco-red"], 0.7),
                    scale_lightness(colors["eco-mid-blue"], 0.7),
                    scale_lightness(colors["eco-mid-blue"], 0.7),
                    scale_lightness(colors["eco-green"], 0.7),
                    scale_lightness(colors["eco-green"], 0.7),
                    scale_lightness(colors["eco-gray"], 0.7),
                    scale_lightness(colors["eco-gray"], 0.7),
                ],
            ),
        ),
        facet=alt.Facet(
            "supergroup_name:N",
            columns=4,
            spacing={"row": 20, "column": -45},
            title=None,
            header=alt.Header(labelColor=colors['eco-gray'],labelFontSize=11)
        ),
        shape=alt.Shape(
            "region:N",
            legend=alt.Legend(
                title="",
                labelLimit=170,
                orient="bottom",
                columns=3,
                symbolFillColor=colors["eco-gray"],
                symbolStrokeColor=scale_lightness(colors["eco-gray"], 0.7),
                offset=30,
            ),
            scale=alt.Scale(
                range=[
                    "triangle-right",
                    "cross",
                    "circle",
                    "triangle-up",
                    "diamond",
                    "triangle-down",
                    "triangle-left",
                    "M0,.5L.6,.8L.5,.1L1,-.3L.3,-.4L0,-1L-.3,-.4L-1,-.3L-.5,.1L-.6,.8L0,.5Z",
                    "square",
                ]
            ),
        ),
        opacity=alt.condition(selection | selection2, alt.value(0.8), alt.value(0.1)),
    )
    .transform_calculate(
        tooltip="datum.region+' 🏨 '+datum.local_authorities+' 📊 X: '+datum.x_axis_imd_rank+'. Y: '+datum.y_axis_allcat_rank+'.'"
    )
    .add_selection(selection)
    .add_selection(selection2)
)
points1 = base.mark_point(size=90).encode()

layer1 = points1.properties(width=130,height=100)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1